In [1]:
from mip import *

In [2]:
inst01 = {
    'm': 2,
    'n': 6,
    'l': [15, 10],
    's': [3, 2, 6, 5, 4, 4],
    'min_dist': 4,
    'max_dist': 40,
    'time': 5,
    'at_least_one': 1,
    'distances': [[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]
}

In [3]:
instance = inst01

In [4]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])
max_distance = np.max(instance["distances"], axis=1).sum()

# deposit = n + 1

# model = Model(solver_name=CBC)
model = Model() # uses Gurobi as default

# x[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
x = model.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="x")

# y[i] is the distance traveled by the courier i
y = model.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i, j] is 1 if the courier i carries the package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

# z is a path counter where z[i, j] is the number of the node j in the path of the courier i
z = model.add_var_tensor((m, n + 1), var_type=INTEGER, lb=0, ub=max_distance, name="z")

# M is a sufficiently large value for the path counter
M = n + 3

# The counter is 0 for the deposit
model += z[:, n] == 0

for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n): # Note that j2 cannot be n + 1 (i.e. the deposit)
            # if x[i, j1, j2] = 1, then z[i, j2] = z[i, j1] + 1

            model += z[i, j2] - z[i, j1] <= M * (1 - x[i, j1, j2]) + 1
            model += z[i, j1] - z[i, j2] <= M * (1 - x[i, j1, j2]) - 1


for i in range(m):
    model += np.sum(x[i, n, :]) == 1
    model += np.sum(x[i, :, n]) == 1


# t[i, j] is 1 if the courier i carries the package j
for i in range(m):
    for j in range(n):
        model += np.sum(x[i, j, :]) == t[i, j]

# All packages are carried by exactly one courier
for j in range(n):
    model += np.sum(t[:, j]) == 1

# If a courier arrives to a package, it has to leave from that package
for i in range(m):
    for j in range(n):
        courier_arrives = np.sum(x[i, :, j])
        courier_leaves = np.sum(x[i, j, :])
        # (not arrives) or leaves
        model += (1 - courier_arrives) + courier_leaves >= 1


"""# Each node must be a middle stop exactly once
for j in range(n):
    model += np.sum(x[i, j, :]) == 1
    model += np.sum(x[i, :, j]) == 1"""

# Each courier does not exceed its capacity
for i in range(m):
    model += (t[i, :] @ s) <= l[i]

# y[i] is the distance traveled by the courier i
for i in range(m):
    model +=  np.sum(x[i, :, :] * np.array(instance["distances"])) == y[i]

# You cannot travel from a node to itself
for i in range(m):
    for j in range(n):
        model += x[i, j, j] == 0


v = model.add_var(name="v", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += v >= y[i]

model.threads = -1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [5]:
print(model.threads)

-1


In [6]:
model.objective = minimize(v)

model.optimize()

Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 222 rows, 127 columns and 876 nonzeros
Model fingerprint: 0x220c7745
Variable types: 0 continuous, 127 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 50 rows and 26 columns
Presolve time: 0.01s
Presolved: 172 rows, 101 columns, 730 nonzeros
Variable types: 0 continuous, 101 integer (88 binary)
Found heuristic solution: objective 18.0000000

Root relaxation: objective 7.222222e+00, 56 iterations, 0.00 seconds (0.00 work units)

    Nodes 

<OptimizationStatus.OPTIMAL: 0>

In [7]:
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            if x[i, j1, j2].x == 1:
                print(f"Courier {i} travels from {j1} to {j2}")

Courier 0 travels from 0 to 6
Courier 0 travels from 2 to 0
Courier 0 travels from 3 to 2
Courier 0 travels from 6 to 3
Courier 1 travels from 1 to 4
Courier 1 travels from 4 to 5
Courier 1 travels from 5 to 6
Courier 1 travels from 6 to 1


In [8]:
def find_next(i, node):
    for j in range(n + 1):
        if x[i, node, j].x == 1:
            return j


for i in range(m):
    steps = [n]
    current_node = n
    
    current_node = find_next(i, current_node)
    steps.append(current_node)
    while current_node != n:
        current_node = find_next(i, current_node)
        steps.append(current_node)
    
    print(f'Courier {i}:', ', '.join([str(node) for node in steps]))


Courier 0: 6, 3, 2, 0, 6
Courier 1: 6, 1, 4, 5, 6


In [9]:
print([y[i].x for i in range(m)])

[14.0, 14.0]


In [10]:
print(model.objective_value)

14.0


In [11]:
print([(i, j) for j in range(n) for i in range(m) if t[i, j].x == 1])

[(0, 0), (1, 1), (0, 2), (0, 3), (1, 4), (1, 5)]
